In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hammadjavaid/vehicle-object-detection-dataset-5-classes")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/vehicle-object-detection-dataset-5-classes


In [2]:
import os

dataset_path = "/kaggle/input/vehicle-object-detection-dataset-5-classes"

# Check if the directory exists
if os.path.exists(dataset_path):
    print(f"Contents of '{dataset_path}':")
    for item in os.listdir(dataset_path):
        print(item)
else:
    print(f"Error: Directory '{dataset_path}' not found.")

Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes':
vehicles


In [3]:
import os

# First, let's check what's actually in the dataset path
print(f"Exploring dataset structure in: {dataset_path}\n")

# Function to explore directory structure
def explore_directory(path, max_depth=5, current_depth=0):
    if current_depth > max_depth:
        return
    
    try:
        items = os.listdir(path)
        indent = "  " * current_depth
        print(f"{indent}Contents of '{path}':")
        
        for item in items:
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                print(f"{indent}  📁 {item}/")
                if current_depth < max_depth:
                    explore_directory(item_path, max_depth, current_depth + 1)
            else:
                print(f"{indent}  📄 {item}")
    except Exception as e:
        print(f"{indent}Error accessing {path}: {e}")

explore_directory(dataset_path, max_depth=2)

# Now try to find the correct dataset subdirectory
possible_paths = [
    os.path.join(dataset_path, 'dataset'),
    dataset_path,  # Sometimes the dataset folder IS the root
]

dataset_sub_path = None
for path in possible_paths:
    if os.path.exists(path):
        # Check if this path has 'images' and 'labels' folders
        contents = os.listdir(path)
        if 'images' in contents and 'labels' in contents:
            dataset_sub_path = path
            print(f"\n✅ Found dataset directory: {dataset_sub_path}")
            break

if dataset_sub_path is None:
    # If still not found, search for images and labels folders
    for root, dirs, files in os.walk(dataset_path):
        if 'images' in dirs and 'labels' in dirs:
            dataset_sub_path = root
            print(f"\n✅ Found dataset directory: {dataset_sub_path}")
            break

if dataset_sub_path:
    print(f"\nContents of dataset directory:")
    for item in os.listdir(dataset_sub_path):
        print(f"  - {item}")
else:
    print("\n❌ Could not find dataset directory with 'images' and 'labels' folders")


Exploring dataset structure in: /kaggle/input/vehicle-object-detection-dataset-5-classes

Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes':
  📁 vehicles/
  Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles':
    📄 data.yaml
    📁 val/
    Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles/val':
      📁 labels/
      📁 images/
    📁 test/
    Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles/test':
      📁 labels/
      📁 images/
    📁 train/
    Contents of '/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles/train':
      📁 labels/
      📁 images/

✅ Found dataset directory: /kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles/val

Contents of dataset directory:
  - labels
  - images


In [8]:
!pip install ultralytics

In [9]:
import ultralytics

print(f"Ultralytics version: {ultralytics.__version__}")

Ultralytics version: 8.4.7


In [10]:
!nvidia-smi

Sun Jan 25 19:12:09 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             28W /   70W |     257MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
import yaml
from pathlib import Path

# Dataset root (read-only)
DATASET_DIR = Path("/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles")

# Writable directory
WORKING_DIR = Path("/kaggle/working")

# ---- Read existing data.yaml ----
original_yaml = DATASET_DIR / "data.yaml"

if not original_yaml.exists():
    raise FileNotFoundError("❌ data.yaml not found in dataset directory")

with open(original_yaml, "r") as f:
    original_data = yaml.safe_load(f)

# Extract class info
classes = original_data.get("names", [])
nc = original_data.get("nc", len(classes))

if not classes or nc == 0:
    raise ValueError("❌ No classes found in original data.yaml")

# ---- Create new YOLO config ----
yolo_data_config = {
    "path": str(DATASET_DIR.resolve()),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": nc,
    "names": classes
}

# Save new YAML
yolo_yaml_path = WORKING_DIR / "data_yolo.yaml"
with open(yolo_yaml_path, "w") as f:
    yaml.dump(yolo_data_config, f, sort_keys=False)

# Print confirmation
print(f"✅ data_yolo.yaml file saved to: {yolo_yaml_path}\n")
print("Content of data_yolo.yaml:")
print(yolo_yaml_path.read_text())


✅ data_yolo.yaml file saved to: /kaggle/working/data_yolo.yaml

Content of data_yolo.yaml:
path: /kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles
train: train/images
val: val/images
test: test/images
nc: 5
names:
- bus
- car
- pickup
- truck
- van



In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8n model
model = YOLO('yolov8n.pt')

# Train the model
print("Initiating model training...")
model.train(data=str(yolo_yaml_path), epochs=15, imgsz=640, batch=16 ,device=0,  cache=True,  augment=True )
print("Model training completed.")

Initiating model training...
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data_yolo.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Paths
WORKING_DIR = Path("/kaggle/working")
best_model_path = WORKING_DIR / "runs/detect/train3/weights/best.pt"

# Load the trained model
print(f"📂 Loading model: {best_model_path}")
model = YOLO(str(best_model_path))

# Validate
print("\n🔍 Validating YOLOv8 model...")
metrics = model.val()

# Display all metrics
print(f"\n📊 Validation Results:")
print(f"   mAP50:     {metrics.box.map50:.4f}")
print(f"   mAP50-95:  {metrics.box.map:.4f}")
print(f"   Precision: {metrics.box.mp:.4f}")
print(f"   Recall:    {metrics.box.mr:.4f}")

In [7]:
import yaml
from pathlib import Path

# Dataset root (read-only)
DATASET_DIR = Path("/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles")

# Writable directory
WORKING_DIR = Path("/kaggle/working")

# ---- Read existing data.yaml ----
original_yaml = DATASET_DIR / "data.yaml"

if not original_yaml.exists():
    raise FileNotFoundError("❌ data.yaml not found in dataset directory")

with open(original_yaml, "r") as f:
    original_data = yaml.safe_load(f)

# Extract class info
classes = original_data.get("names", [])
nc = original_data.get("nc", len(classes))

if not classes or nc == 0:
    raise ValueError("❌ No classes found in original data.yaml")

# ---- Create new YOLO config ----
yolo_data_config = {
    "path": str(DATASET_DIR.resolve()),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": nc,
    "names": classes
}

# Save new YAML
yolo_yaml_path = WORKING_DIR / "data_yolo.yaml"
with open(yolo_yaml_path, "w") as f:
    yaml.dump(yolo_data_config, f, sort_keys=False)

# Print confirmation
print(f"✅ data_yolo.yaml file saved to: {yolo_yaml_path}\n")
print("Content of data_yolo.yaml:")
print(yolo_yaml_path.read_text())


✅ data_yolo.yaml file saved to: /kaggle/working/data_yolo.yaml

Content of data_yolo.yaml:
path: /kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles
train: train/images
val: val/images
test: test/images
nc: 5
names:
- bus
- car
- pickup
- truck
- van



In [8]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8n model
model = YOLO('yolov10n.pt')

# Train the model
print("Initiating model training...")
model.train(data=str(yolo_yaml_path), epochs=15, imgsz=640, batch=16 ,device=0,  cache=True,  augment=True )
print("Model training completed.")

Initiating model training...
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data_yolo.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

In [ ]:
print("🔍 Validating YOLOv10 model...")
metrics = model.val()
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")


In [ ]:
import yaml
from pathlib import Path

# Dataset root (read-only)
DATASET_DIR = Path("/kaggle/input/vehicle-object-detection-dataset-5-classes/vehicles")
# Writable directory
WORKING_DIR = Path("/kaggle/working")

# ---- Read existing data.yaml ----
original_yaml = DATASET_DIR / "data.yaml"
if not original_yaml.exists():
    raise FileNotFoundError("❌ data.yaml not found in dataset directory")

with open(original_yaml, "r") as f:
    original_data = yaml.safe_load(f)

# Extract class info
classes = original_data.get("names", [])
nc = original_data.get("nc", len(classes))

if not classes or nc == 0:
    raise ValueError("❌ No classes found in original data.yaml")

# ---- Create new YOLO config ----
yolo_data_config = {
    "path": str(DATASET_DIR.resolve()),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": nc,
    "names": classes
}

# Save new YAML
yolo_yaml_path = WORKING_DIR / "data_yolo.yaml"
with open(yolo_yaml_path, "w") as f:
    yaml.dump(yolo_data_config, f, sort_keys=False)

# Print confirmation
print(f"✅ data_yolo.yaml file saved to: {yolo_yaml_path}\n")
print("Content of data_yolo.yaml:")
print(yolo_yaml_path.read_text())

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv11n model
model = YOLO('yolo11n.pt')

# Train the model
print("🚀 Initiating YOLOv11 model training...")
results = model.train(
    data=str(yolo_yaml_path), 
    epochs=20, 
    imgsz=640, 
    batch=16,
    device=0,
    cache=True,
    augment=True,
    project='yolov11_training',
    name='vehicle_detection'
)
print("✅ YOLOv11 model training completed.")

In [ ]:
print("🔍 Validating YOLOv11 model...")
metrics = model.val()
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")